In [1]:
import cv2
import numpy as np
import math
import random
import keyboard

import time

In [3]:

def place_car(img, X,Y,voiture):
    hight=math.ceil(voiture.shape[0]/2)
    width=math.ceil(voiture.shape[1]/2)
    if (X-width>0 and Y-hight>0 and X+width<img.shape[1] and Y+hight<img.shape[0]) :
        img[Y-hight:Y + hight, X-width:X + width, :] = voiture 
    return img
def remove_car(img, X,Y,voiture):
    game_over=False
    hight=math.ceil(voiture.shape[0]/2)
    width=math.ceil(voiture.shape[1]/2)
    if (X-width>0 and Y-hight>0 and X+width<img.shape[1] and Y+hight<img.shape[0]) :
        img[Y-hight:Y + hight, X-width:X + width, :] = np.zeros((voiture.shape[0],voiture.shape[1],3)) 
    return img
def place_meteore(img,liste_meteor):
    for i in liste_meteor:
        img=place_car(img,i.posX,i.posY,i.img)
    return img
def remove_meteore(img,liste_meteor):
    for i in liste_meteor:
        img=remove_car(img,i.posX,i.posY,i.img)
    return img




In [4]:
class Meteore:
    def __init__(self, img,posX,posY,moveX,moveY,radius):
        self.img = img
        self.posX=posX
        self.posY=posY
        self.moveX=moveX
        self.moveY=moveY
        self.radius=radius
    def update(self):
        self.posX+=self.moveX
        self.posY+=self.moveY
    def greet(self):
        print(f"Hello, {self.name}!")


In [25]:
liste_meteor=[]
for i in range(1):
    met=np.zeros((2*random.randint(10,50),2*random.randint(10,50),3))
    for w in range(met.shape[0]):
        for h in range(met.shape[1]):
            met[w][h][:]=[255,255,255]
    positiony=random.randint(0,300)
    positionx=random.randint(100,540)
    liste_meteor.append(Meteore(met,positiony,positionx,random.randint(-100,100),random.randint(-100,100),abs(positiony-positionx)))

In [26]:
for i in liste_meteor:
    print(i.posX,i.posY,i.moveX,i.moveY,i.radius)
    print(i.img.shape)
    print("")

59 306 41 95 247
(36, 68, 3)



In [24]:

voiture=np.zeros((100,100,3))
for w in range(voiture.shape[0]):
    for h in range(voiture.shape[1]):
        voiture[w][h][:]=[255,255,255]




obstaclespeed=100

In [29]:
start=True
game_over=False
cap = cv2.VideoCapture(0)
img=np.zeros(shape=(700, 640, 3))
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Error reading frame")
        break
    if game_over:
        img=np.zeros(shape=(700, 640, 3))
        cv2.putText(img, "Game Over", (int(640/2)-100, int(700/2)), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2)
        cv2.imshow('game', img)
        if cv2.waitKey(1) & 0xFF == ord('a'):
            strart=True
            game_over=False
    else:
        
        if start:
            start=False
            posx=int(640/2)
            posy=700-70
            img=np.zeros(shape=(700, 640, 3))
            img=place_car(img,posx,posy,voiture)
            img=place_meteore(img,liste_meteor)
            cv2.imshow('game', img)
            
            
            
        else:      
            img=remove_car(img,posx,posy,voiture)
            img=remove_meteore(img,liste_meteor)
            if keyboard.is_pressed('g'):
                posx-=10
            if keyboard.is_pressed('h'):
                posx+=10
            
            for i in range(len(liste_meteor)):
                liste_meteor[i].update()
                if liste_meteor[i].posX<0 or liste_meteor[i].posX>img.shape[1]:
                    liste_meteor[i].moveX=-liste_meteor[i].moveX
                if liste_meteor[i].posY<0 or liste_meteor[i].posY>1000:
                    liste_meteor[i].posY=0
            img=place_car(img,posx,posy,voiture)
            img=place_meteore(img,liste_meteor)
            cv2.imshow('game', img)
            

    # Read the frame from the webcam
    
    
    


    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [14]:
cap.release()
cv2.destroyAllWindows()